In [1]:
from IPython.display import Image, display

In [20]:
from typing import List,  Annotated, TypedDict, operator, Literal
from pydantic import BaseModel, Field

from langchain.chat_models import init_chat_model
from langchain_core.tools import tool
from langgraph.graph import MessagesState
from langchain_tavily import TavilySearch

from langgraph.types import Command, Send
from langgraph.graph import START, END, StateGraph


## LLM 
llm = init_chat_model(
    model="openai:o3-mini",
    temperature=0.0
)

## Tools 
tavily_search_tool = TavilySearch(
    max_results=5,
    topic="general",
)

@tool
class Section(BaseModel):
    name: str = Field(
        description="Name for this section of the report.",
    )
    description: str = Field(
        description="Research scope for this section of the report.",
    )
    content: str = Field(
        description="The content of the section."
    )

@tool
class Sections(BaseModel):
    sections: List[Section] = Field(
        description="Sections of the report.",
    )

@tool
class Introduction(BaseModel):
    name: str = Field(
        description="Name for the report.",
    )
    content: str = Field(
        description="The content of the introduction, giving an overview of the report."
    )

@tool
class Conclusion(BaseModel):
    name: str = Field(
        description="Name for the conclusion of the report.",
    )
    content: str = Field(
        description="The content of the conclusion, summarizing the report."
    )

@tool
class Done(BaseModel):
      """Tool to signal that the work is complete."""
      done: bool


## State
class ReportStateOutput(TypedDict):
    final_report: str # Final report

class ReportState(MessagesState):
    sections: list[Section] # List of report sections 
    completed_sections: Annotated[list, operator.add] # Send() API key
    final_report: str # Final report

class SectionState(MessagesState):
    section: Section # Report section  
    completed_sections: list[Section] # Final key we duplicate in outer state for Send() API

class SectionOutputState(TypedDict):
    completed_sections: list[Section] # Final key we duplicate in outer state for Send() API

## Supervisor

SUPERVISOR_INSTRUCTIONS = """
You are scoping research for a report based on a user-provided topic.

### Your responsibilities:

1. **Clarify the Topic**  
   Engage with the user to clarify their intent. Ask follow-up questions to fully understand the topic, goals, constraints, and any preferences for the report.

2. **Gather Background Information**  
   Use the `tavily_search_tool` to collect relevant information about the topic. Use multiple focused queries to build context from different angles.

3. **Define Report Structure**  
   Once you understand the topic:
   - Use the `Sections` tool to define a structured outline of the report.
   - Each section should include a clear name and scope description.
   - Leave the content preview empty because the research agent will fill it in.
   - Ensure sections are scoped to be independently researchable.

4. **Assemble the Final Report**  
   When all sections are returned:
   - Use the `Introduction` tool to generate a clear and informative introduction.
   - Use the `Conclusion` tool to summarize key insights and close the report.

5. **Finish the Workflow**  
   When the final report is complete, call the `Done` tool to signal that the work is finished.

### Additional Notes:
- You are a reasoning model. Think through problems step-by-step before acting.
- Use your tools wisely—especially the search tool—to improve research quality.
- Maintain a clear, informative, and professional tone throughout."""

RESEARCH_INSTRUCTIONS = """
You are a researcher responsible for completing a specific section of a report.

### Your goals:

1. **Understand the Section Scope**  
   Begin by reviewing the section name and description. This defines your research focus. Use it as your objective.

<Section Name>
{section_name}
</Section Name>

<Section Description>
{section_description}
</Section Description>

2. **Research the Topic**  
   Use the `tavily_search_tool` to gather relevant information and evidence. Search iteratively if needed to fully understand the section’s scope.

3. **Use the Section Tool**  
   Once you’ve gathered sufficient context, write a high-quality called the Section tool to write the section. Your content should:
   - `name`: The title of the section
   - `description`: The scope of research you completed 
   - `content`: The completed body of text for the section

---

### Reasoning Guidance

You are a reasoning model. Think through the task step-by-step before writing. Break down complex questions. If you're unsure about something, search again.

- You may reason internally before producing content.
- Your job is not to summarize randomly—it's to **research and synthesize a strong, scoped contribution** to a report.

---

### Notes:
- Do not write introductions or conclusions unless explicitly part of your section.
- Keep a professional, factual tone.
- If you do not have enough information to complete the section, search again or clarify your approach before continuing.
"""

# Tools
supervisor_tools = [tavily_search_tool, Sections, Introduction, Conclusion]
supervisor_tools_by_name = {tool.name: tool for tool in supervisor_tools}

research_tools = [tavily_search_tool, Section]
research_tools_by_name = {tool.name: tool for tool in research_tools}

def supervisor(state: ReportState):
    """LLM decides whether to call a tool or not"""

    # Messages
    messages = state["messages"]

    # Check if we have research completed
    if state.get("sections"):
        research_complete_message = {"role": "user", "content": "Research is complete. Write the introduction and conclusion.\n\n" + "\n\n".join([s.content for s in state["sections"]])}
        messages = messages + [research_complete_message]

    # Invoke LLM
    return {
        "messages": [
            llm.bind_tools(supervisor_tools_by_name).invoke(
                [
                    {"role": "system",
                     "content": SUPERVISOR_INSTRUCTIONS,
                    }
                ]
                + messages
            )
        ]
    }

def supervisor_tools(state: ReportState)  -> Command[Literal["supervisor", "research_team", "__end__"]]:
    """Performs the tool call and sends to the research agent"""

    result = []
    # Get the last message
    for tool_call in state["messages"][-1].tool_calls:
        # Get the tool
        tool = supervisor_tools_by_name[tool_call["name"]]
        # Perform the tool call
        observation = tool.invoke(tool_call["args"])
        # Append to messages 
        result.append([{"role": "tool", 
                        "content": observation, 
                        "name": tool_call["name"], 
                        "tool_call_id": tool_call["id"]}])
        # Update state depending on the tool call 
        if tool_call["name"] == "Sections":
            return Command(goto=[Send("research_team", {"section": s}) for s in state['sections']])
        if tool_call["name"] == "Introduction":
            return Command(goto="supervisor", update={"sections": [tool_call["args"]] + state["sections"], "messages": result})
        if tool_call["name"] == "Conclusion":
            return Command(goto="supervisor", update={"sections": state["sections"] + [tool_call["args"]], "messages": result})
        else:
            return Command(goto="supervisor", update={"messages": result})

def research_agent(state: SectionState):
    """LLM decides whether to call a tool or not"""

    return {
        "messages": [
            # Enforce tool calling to either perform more search or call the Section tool to write the section
            init_chat_model("openai:o3", tool_choice="required", temperature=0.0).bind_tools(research_tools).invoke(
                [
                    {"role": "system",
                     "content": RESEARCH_INSTRUCTIONS.format(section_name=state["section"].name, section_description=state["section"].description)
                    }
                ]
                + state["messages"]
            )
        ]
    }

def research_agent_tools(state: SectionState) -> Command[Literal["supervisor", "research_agent"]]:
    """Performs the tool call and route to supervisor or continue the research loop"""

    result = []
    # Get the last message
    for tool_call in state["messages"][-1].tool_calls:
        # Get the tool
        tool = research_tools_by_name[tool_call["name"]]
        # Perform the tool call
        observation = tool.invoke(tool_call["args"])
        # Append to messages 
        result.append([{"role": "tool", 
                        "content": observation, 
                        "name": tool_call["name"], 
                        "tool_call_id": tool_call["id"]}])
        # It it wrote the section, send it to the supervisor
        if tool_call["name"] == "Section":
            return Command(goto="supervisor", update={"messages": result, "completed_sections": [tool_call["args"]]})
        # Otherwise continue the research loop
        elif tool_call["name"] == "tavily_search_tool":
            return Command(goto="research_agent", update={"messages": result})
        else:
            return ValueError(f"Tool {tool_call['name']} invalid")

def should_continue(state: ReportState) -> Literal["supervisor_tools", END]:
    """Decide if we should continue the loop or stop based upon whether the LLM made a tool call"""

    messages = state["messages"]
    last_message = messages[-1]

    # If the LLM makes a tool call, then perform an action
    if last_message.tool_calls:
        return "supervisor_tools"

    # Otherwise research is complete and we can assemble the final report
    all_sections = "\n\n".join([s.content for s in state["sections"]])
    return Command(goto=END, update={"final_report": all_sections})

# Research agent workflow
research_builder = StateGraph(SectionState, output=SectionOutputState)
research_builder.add_node("research_agent", research_agent)
research_builder.add_node("research_agent_tools", research_agent_tools)
research_builder.add_edge(START, "research_agent") 

# Build workflow
supervisor_builder = StateGraph(ReportState, input=MessagesState, output=ReportStateOutput)
supervisor_builder.add_node("supervisor", supervisor)
supervisor_builder.add_node("supervisor_tools", supervisor_tools)
supervisor_builder.add_node("research_team", research_builder.compile())

# Flow of the supervisor agent
supervisor_builder.add_edge(START, "supervisor")
supervisor_builder.add_conditional_edges(
    "supervisor",
    should_continue,
    {
        # Name returned by should_continue : Name of next node to visit
        "supervisor_tools": "supervisor_tools",
        END: END,
    },
)

# Compile the supervisor agent
agent = supervisor_builder.compile(name="research_team")
display(Image(agent.get_graph(xray=1).draw_mermaid_png()))

ValueError: Found edge ending at unknown node `supervisor`

In [ ]:
# Build workflow
agent_builder = StateGraph(ReportState, input=MessagesState,output=ReportStateOutput)
agent_builder.add_node("supervisor", supervisor)
agent_builder.add_node("supervisor_tools", supervisor_tools)
agent_builder.add_node("research_agent", research_agent)
agent_builder.add_node("research_agent_tools", research_agent_tools)

agent_builder.add_edge(START, "supervisor")
agent_builder.add_conditional_edges(
    "supervisor",
    should_continue,
    {
        # Name returned by should_continue : Name of next node to visit
        "supervisor_tools": "supervisor_tools",
        END: END,
    },
)

agent = agent_builder.compile(name="research_team")
display(Image(agent.get_graph(xray=1).draw_mermaid_png()))